In [5]:
import pandas as pd
import re
from datetime import datetime
import praw
import pandas as pd
import matplotlib.pyplot as plt



reddit = praw.Reddit(client_id = "QC3sHGjf2WVbx3rZnzJBzw",
                     client_secret = "z7-Qudmgrz4Hj9KXKHT3GYKFiAeg4g",
                     user_agent = "CSE_482_project by Round_Lab_9338")

In [14]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut','Delaware', 'Florida', 'Georgia', 'Hawaii',
          'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
          'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'NewHampshire', 'newjersey', 'NewMexico', 'NewYork',
          'NorthCarolina', 'NorthDakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'RhodeIsland', 'SouthCarolina', 'SouthDakota',
          'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'WestVirginia', 'Wisconsin','Wyoming']
states = list(map(lambda x: x.lower(), states))

# Cities subbreddits are key, and states are the values
cities_to_state = {'birmingham':'alabama',
                   'fairbanks':'alaska',
                   'phoenix': 'arizona', 'tucson': 'arizona',
                   'littlerock':'arkansas', 'fayetteville':'arkansas',
                   'losangeles':'california','sandiego':'california', 'santarosa':'california', 'santabarbara':'california', 'sanfrancisco':'california', 'palmsprings':'california',
                   'coloradosprings':'colorado', 'aurora':'colorado',
                   'hartford':'connecticut',
                   'miami':'flordia', 'orlando': 'flordia', 'jacksonville':'flordia',
                   'atlanta':'georgia', 'augusta':'georgia', 'roswell':'georgia',
                   'honolulu':'hawaii',
                   'boise':'idaho', 'nampa':'idaho',
                   'naperville':'illinois', 'chicago':'illinois',
                   'indianapolis':'indiana', 'bloomington':'indiana',
                   'iowacity':'iowa', 'desmoines':'iowa',
                   'wichita':'kansas', 'kansascity':'kansas',
                   'louisville':'kentucky', 'lexington':'Kentucky',
                   'neworleans':'louisiana', 'shreveport':'louisiana',
                   'frederickmd':'maryland', 'annapolis':'maryland',
                   'boston':'massachusetts', 'salem':'massachusetts',
                   'annarbor':"michigan", "grandrapids":"michigan", 'detroit':"michigan", 'ferndale':'michigan',
                   #couldn't find a subreddit for cities in maine
                   'minneapolis':'minnesota', 'duluth':'minnesota',
                   'jacksonmi':'mississippi', 'tupelo':'mississippi',
                   'bransonmo':'missouri', 'springfield':'missouri',
                   'bozeman':'montana', 'helena':'montana',
                   'omaha':'nebraska', 'kearney':'nebraska',
                   'lasvegas':'nevada', 'sparksnv':'nevada', 'reno':'nevada', 'hendersonnv':'nevada',
                   'portsmouthnh':'newhampshire', 'manchesternh':'newhampshire',
                   'southjersey':'newjersey', 'newark':'newjersey', 'jerseycity':'newjersey',
                   'roswellnm':'newmexico', 'albuquerque':'newmexico',
                   'nyc':'newyork', 'pittsburgh':'newyork', 'buffalo':'newyork', 'columbia':'newyork', 'albany':'newyork',
                   'charlotte':'northcarolina', 'wilmington':'northcarolina', 'greenvillencarolina': 'northcarolina',
                   'fargo':'northdakota', 'bismarck':'northdakota', 'minot':'northdakota',
                   'columbus':'ohio', 'cleveland':'ohio', 'toledo':'ohio', 'cincinnati':'ohio', 'dayton':'ohio', 'akron':'ohio',
                   'tulsa':'oklahoma', 'okc':'oklahoma', #this is oklahoma city subreddit
                   'portland':'oregon', 'portlandor':'oregon', 'bend': 'oregon', 'eugene':'oregon',
                   'philadelphia':'pennsylvania','philly':'pennsylvania', 'pittsburgh':'pennsylvania', 'harrisburg':'pennsylvania', 'erie':'pennsylvania',
                    # rhode island is a city
                   'charleston':'southcarolina', 'myrtlebeach':'southcarolina', 'greenville':'southcarolina',
                   'siouxfalls':'southdakota', 'pierre':'southdakota',
                   'nashville':'tennessee', 'memphis':'tennessee', 'knoxville':'tennessee',
                   'houston':'texas', 'dallas':'texas', 'sanantonio':'texas', 'austin':'texas', 'elpaso':'texas', 'fortworth':'texas', 'lubbock':'texas', 'bedford':'texas',
                   'saltlakecity':'utah', 'provoutah':'utah', 'stgeorge':'utah',
                   'burlington':'vermont', 'montpelier':'vermont',
                   'langley':'virginia', 'richmondva':'virginia',
                   'seattle':'washington', 'seattlewa':'washington',
                   'morgantownwv':'westvirginia', 'parkersburg':'westvirginia',
                   'milwaukee':'wisconsin', 'madisonwi':'wisconsin', 'appleton':'wisconsin',
                   'cheyenne':'wyoming', 'casper':'wyoming'}

cities_subbredit = list(cities_to_state.keys())

In [16]:
# City to state converter.
# https://github.com/zazencodes/city_to_state_dictionary/blob/master/US%20City%20to%20State%20dictionary.ipynb
saved_tesla = pd.read_csv("tesla_reddit_data3.csv")

def findstate(post_sub):
    sub_name = str(post_sub).lower()
    if sub_name in states:
        return sub_name
    elif sub_name in cities_to_state:
            return cities_to_state[sub_name]
    else:
        return "unkown"
        
def getpost(subred, find, time):
    subreddit = reddit.subreddit(subred)
    unique_posts_id = set()
    posts = []
    for post in subreddit.search(find, time_filter=time):
        # print(subred)
        if post.id not in unique_posts_id:
            unique_posts_id.add(post.id)
            posts.append({
                'id': post.id,
                'title': post.title,
                'selftext': post.selftext,
                'subreddit': post.subreddit.display_name,
                'url': post.url,
                'created_utc': post.created_utc,
                'upvotes': post.score,
                'comments': post.num_comments,
                'state': findstate(post.subreddit.display_name)
            })
    return pd.DataFrame(posts)

def get_state_post(time):
    new_tesla = pd.DataFrame([])
    states_col = []
    for state in states:
        post = getpost(state, 'tesla', time)
        if post.shape[0] !=0:
            new_tesla = pd.concat([new_tesla, post], axis = 0)

    new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    return new_tesla

def get_city_post(time):
    new_tesla = pd.DataFrame([])
    states_col = []
    for city in cities_subbredit:
        print(city, end ='\t')
        post = getpost(city, 'tesla', time)
        if post.shape[0] !=0:
            new_tesla = pd.concat([new_tesla, post], axis = 0)

    new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    return new_tesla



# Scratch work below

get_state_post() and get_cities_post() just need in a time variable to run: 'all', 'year','month', 'week' or 'day'

In [22]:
new_post_state = get_state_post('day')
print(len(new_post_state))

12


C:\Users\olivi\AppData\Local\Temp\ipykernel_18128\4143237611.py:43: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


In [25]:
new_post_cites = get_city_post('day')
print('\n',len(new_post_cites))

birmingham	fairbanks	phoenix	tucson	littlerock	fayetteville	losangeles	sandiego	santarosa	santabarbara	sanfrancisco	palmsprings	coloradosprings	aurora	hartford	miami	orlando	jacksonville	atlanta	augusta	roswell	honolulu	boise	nampa	naperville	chicago	indianapolis	bloomington	iowacity	desmoines	wichita	kansascity	louisville	lexington	neworleans	shreveport	frederickmd	annapolis	boston	salem	annarbor	grandrapids	detroit	ferndale	minneapolis	duluth	jacksonmi	tupelo	bransonmo	springfield	bozeman	helena	omaha	kearney	lasvegas	sparksnv	reno	hendersonnv	portsmouthnh	manchesternh	southjersey	newark	jerseycity	roswellnm	albuquerque	nyc	pittsburgh	buffalo	columbia	albany	charlotte	wilmington	greenvillencarolina	fargo	bismarck	minot	columbus	cleveland	toledo	cincinnati	dayton	akron	tulsa	okc	portland	portlandor	bend	eugene	philadelphia	philly	harrisburg	erie	charleston	myrtlebeach	greenville	siouxfalls	pierre	nashville	memphis	knoxville	houston	dallas	sanantonio	austin	elpaso	fortworth	lubbock	bed

C:\Users\olivi\AppData\Local\Temp\ipykernel_18128\4143237611.py:55: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  new_tesla['created_utc'] = new_tesla['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


In [29]:
post_to_save = pd.DataFrame()

In [31]:
post_to_save = pd.concat([post_to_save, new_post_state], axis = 0)
post_to_save = pd.concat([post_to_save, new_post_cites], axis = 0)
post_to_save = post_to_save.drop_duplicates(subset=['id'])
post_to_save.shape

(30, 9)

In [33]:
post_to_save

,id,title,selftext,subreddit,url,created_utc,upvotes,comments,state
0,1jpab5p,Anyone have a Tesla model y in CT? What’s your...,Nj resident here. Wife and I are thinking abou...,Connecticut,https://www.reddit.com/r/Connecticut/comments/...,2025-04-01 23:34:13,0,5,connecticut
0,1jou5hp,"Advice please! SSA phones down, any other way ...",,florida,https://www.reddit.com/r/florida/comments/1jou...,2025-04-01 12:17:56,41,59,florida
0,1jolm15,National protests,This Saturday there will be a nationwide prote...,Iowa,https://www.reddit.com/r/Iowa/comments/1jolm15...,2025-04-01 02:56:07,57,9,iowa
0,1jpa7y3,First ever concert…,My very first concert was in Baton Rouge. The ...,Louisiana,https://www.reddit.com/r/Louisiana/comments/1j...,2025-04-01 23:30:03,2,0,louisiana
0,1jozxoe,Man keys Tesla Cybertruck at West Fargo Costco,,northdakota,https://www.valleynewslive.com/2025/03/30/caug...,2025-04-01 16:29:26,67,141,northdakota
0,1jolc7f,Where are the April 5 protests? Is your town ...,We will be in south east Ohio area on Saturday...,Ohio,https://www.reddit.com/r/Ohio/comments/1jolc7f...,2025-04-01 02:41:39,0,18,ohio
0,1joi4ct,Tesla Model 3 crashes and causes huge traffic ...,Just passed tons of traffic on 283 in Lancaste...,Pennsylvania,https://www.reddit.com/r/Pennsylvania/comments...,2025-04-01 00:00:16,255,46,pennsylvania
0,1jok7f1,"US attorney general announces federal charges,...",,texas,https://kdvr.com/news/local/us-attorney-genera...,2025-04-01 01:44:07,173,107,texas
1,1joxlg5,Volunteers step up to save Gulf Coast wildlife...,Port A is a Texas treasure! DOGE cuts are maki...,texas,https://www.chron.com/gulf-coast/article/texas...,2025-04-01 14:55:11,55,2,texas
0,1jp9lo4,"Sat., April 5th Hands Off Our Democracy! Prote...",,Virginia,https://i.redd.it/o28tzmhacyre1.jpeg,2025-04-01 23:02:29,32,1,virginia


## For add new data to CSV
Be careful not to erase old data

In [41]:
saved_post= pd.read_csv('tesla_only_geo_data.csv')

In [43]:
post_to_save = pd.concat([post_to_save, saved_post], axis = 0)
post_to_save = post_to_save.drop_duplicates(subset=['id'])
post_to_save.shape

(6235, 9)

In [45]:
post_to_save.to_csv('tesla_only_geo_data.csv',index=False)

In [224]:
#df.to_csv('file_name.csv',index=False)